In [4]:
!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

     |████████████████████████████████| 2.9 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 768 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 880 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 2.6 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f43bc280a4e6c5944a942a669a8e5124dfcd7932a983dd5285005fe556b6c97b
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
     |████████████████████████████████| 50 kB 2.6 MB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-linux_x86_64.whl size=172291 sha256=4d2b007267be89a90cd30d461486c1870513524e6e8fa649d31e2f5aee24a20e
  Stored in directory: /root/.cache/pip/wheels/d7/0c/76/042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Succ

In [32]:
!pip install torchaudio==0.8.1

  Using cached torchaudio-0.8.1-cp38-cp38-manylinux1_x86_64.whl (1.9 MB)
  Using cached torch-1.8.1-cp38-cp38-manylinux1_x86_64.whl (804.1 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.0
    Uninstalling torchaudio-0.12.0:
      Successfully uninstalled torchaudio-0.12.0


In [9]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# tokenize
input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest").input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)


Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

...

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset librispeech_asr_dummy downloaded and prepared to /root/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr_dummy/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc. Subsequent calls will reuse this data.


It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [10]:
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
from jiwer import wer


librispeech_eval = load_dataset("librispeech_asr", "clean", split="test")

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

def map_to_pred(batch):
    input_values = processor(batch["audio"]["array"], return_tensors="pt", padding="longest").input_values
    with torch.no_grad():
        logits = model(input_values.to("cuda")).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

result = librispeech_eval.map(map_to_pred, batched=True, batch_size=1, remove_columns=["audio"])

print("WER:", wer(result["text"], result["transcription"]))


KeyboardInterrupt: 

In [1]:
from jiwer import wer
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and tokenizer

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import torch
import torch.nn as nn
model.lm_head = nn.Sequential(
                nn.Flatten(),
                nn.Linear(249*768,1),
                nn.Sigmoid()
)

In [3]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [4]:
import numpy as np
import librosa
from helper_code import *
from tqdm import tqdm
import pywt
from sklearn.preprocessing import MinMaxScaler
import pickle as pk
import glob
import os
import time
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import sys
import random
from tqdm import tqdm
from tools.utils import  get_class_distribution,get_class_distribution2,get_logger,  murmur_score,compute_weighted_accuracy
sys.path.append("..")
from reader.data_reader_physionet import myDataLoader, myDataset
import gc
#from torch.utils.tensorboard import SummaryWriter
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
#from sklearn.ensemble import RandomForestClassifier
from models import *
from get_feature import feature_extract_raw, get_feature_one, feature_extract_wavelet_raw
import pickle as pk
from sklearn.metrics import f1_score
import numpy as np, os, sys
from helper_code import *
from team_code import load_challenge_model, save_challenge_model
from tqdm import tqdm

In [11]:
gc.collect()
torch.cuda.empty_cache()
GPU_NUM = 4 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

In [6]:
data_folder = '/Data1/physionet/data_split/murmur/train'
dev_folder = '/Data1/physionet/data_split/murmur/test'
verbose = 1

In [7]:
def get_features(data_folder, patient_files_trn, raw = True,rand = False) :
    features = dict()

    features['pw_raw'] = []
    features['raw'] = []
    features['mel1'] = []
    murmurs = np.empty((0,2))#3))
    labels = {}
    
    murmur_classes = ['Present', 'Absent'] # ,'Unknown'
    num_murmur_classes = len(murmur_classes)

    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in tqdm(range(num_patient_files)):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        if get_murmur(current_patient_data) != 'Present' :
            murmur_location_information = 'nan'
        else :
            murmur_location_information = current_patient_data.split('\n')[-15].split()[2].split('+')
        
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)
            filename_location = recording_file.split('_')[1].split('.')[0]
            
            if raw :
                raw1, n_samp = feature_extract_raw(filename,samp_sec=20)
#                raw2, n_samp = feature_extract_wavelet_raw(filename,samp_sec=20)
                features['raw'].extend(raw1)
#                features['pw_raw'].extend(raw2)
            # Extract melspec
            else :
                mel1 = feature_extract_raw_melspec(filename,rand)  # np > shape (n_samp,n_mels,time)(n,64,201)
                
                
                n_samp = len(mel1)
                features['mel1'].extend(mel1)


            # Extract labels and use one-hot encoding.
            
            # 클래스 일단 3개지만 학습 데이터에선 loc 아니면 absent로 봄
            
            current_murmur = np.zeros((1,num_murmur_classes), dtype=int)  
#            if (filename_location in murmur_location_information)  :
#                murmur = get_murmur(current_patient_data)
#            else :
#                murmur = 'Absent'
            murmur = get_murmur(current_patient_data)
            if murmur in murmur_classes:
                j = murmur_classes.index(murmur)
                current_murmur[0][j] = 1
            elif murmur == 'Unknown' :
                current_murmur[0][0] = 0
                current_murmur[0][1] = 1
            current_murmur = np.repeat(current_murmur,n_samp,0)
            murmurs = np.concatenate((murmurs,current_murmur),axis=0)
            
    
    labels['murmur'] = murmurs
    labels['outcome'] = []
#    features['pw_raw'] = np.array(features['pw_raw'])
    features['raw'] = np.array(features['raw'])
    
    
    return features, labels

In [8]:
a = [1,0]
b = [0.5,0.5]
print(b.index(0.5))

0


# wav2vec2

In [9]:
# Find the patient data files.
patient_files = find_patient_files(data_folder)
dev_patient_files = find_patient_files(dev_folder)

num_patient_files = len(patient_files)
dev_num_patient_files = len(dev_patient_files)

if num_patient_files==0:
    raise Exception('No data was provided.')


murmur_classes = ['Present', 'Absent'] # ,'Unknown'
num_murmur_classes = len(murmur_classes)


# Extract the features and labels.
if verbose >= 1:
    print('Extracting features and labels from the Challenge data...')

features, labels = get_features(data_folder, patient_files)
features_dev, labels_dev = get_features(dev_folder, dev_patient_files)

Extracting features and labels from the Challenge data...


100%|████████████████████████████████████████| 191/191 [00:00<00:00, 485.34it/s]


In [11]:
features['raw'].shape[0]

2545

In [12]:
labels['murmur'].shape

(2545, 2)

In [13]:
#torch.cuda.empty_cache()

In [12]:
#torch.cuda.empty_cache()


log_dir = './log/'
project = 'toy0'
logger = get_logger(log_dir + '/' + project)

#train
label_mur = np.empty(len(labels['murmur']))
for i,l in enumerate(labels['murmur']) :
    try :
        label_mur[i] = int(list(l).index(1))
    except :
        label_mur[i] = 0.5
label_out = np.empty(len(labels['outcome']))
for i,l in enumerate(labels['outcome']) :
    label_out[i] = int(list(l).index(1))
label = {'murmur':label_mur, 'outcome':label_out}

#dev
label_mur_dev = np.empty(len(labels_dev['murmur']))
for i,l in enumerate(labels_dev['murmur']) :
    try :
        label_mur_dev[i] = int(list(l).index(1))
    except :
        label_mur_dev[i] = 0.5
label_out_dev = np.empty(len(labels_dev['outcome']))
for i,l in enumerate(labels_dev['outcome']) :
    label_out_dev[i] = int(list(l).index(1))
label_dev = {'murmur':label_mur_dev, 'outcome':label_out_dev}

#    class_count = [i for i in get_class_distribution(label_mur).values()]
#    class_weights = 1./torch.tensor(class_count, dtype=torch.float)

class_weights = torch.tensor([5/9,4/9],dtype=torch.float)

dataset_train = myDataset(features, label,mode ='murmur')
dataloader_train = myDataLoader(dataset=dataset_train,
                                batch_size=32,
                                shuffle=True,
                                num_workers=0)
all_file_train = len(dataloader_train)
if verbose >= 1:
    print("- {} training samples".format(len(dataset_train)))
    print("- {} training batches".format(len(dataloader_train)))
dataset_dev = myDataset(features_dev, label_dev,mode ='murmur')
dataloader_dev = myDataLoader(dataset=dataset_dev,
                                batch_size=32,
                                shuffle=True,
                                num_workers=0)
all_file_dev = len(dataloader_dev)
if verbose >= 1:
    print("- {} testing samples".format(len(dataset_dev)))
    print("- {} testing batches".format(len(dataloader_dev)))


#nnet = LCNN(mode = 'murmur')
#nnet = nnet.cuda()
nnet = model.cuda()
softmax = nn.Softmax(dim=1)
sig = nn.Sigmoid()

optimizer = optim.Adam(nnet.parameters(), lr=1e-5)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.97)

#criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
criterion = nn.BCELoss()

# Train the model.
if verbose >= 1:
    print('Training model...')


for iter_ in range(20):  # args.end_iter
    start_time = time.time()
    running_loss = 0.0

    train_epoch_mur_f1 = 0.0

    nnet.train()

    pre_train = []
    label_train = []

    for audio_feature, data_label_torch in tqdm(dataloader_train): 

#            alpha =np.random.randint(4,10)/10
#            lam = np.random.beta(alpha,alpha)
#            audio_feature =  torch.cat([audio_feature,lam*audio_feature + (1.0-lam)*audio_feature2], dim=1)
#            data_label_torch_mur = torch.cat([data_label_torch_mur,lam*data_label_torch_mur + (1.0-lam)*data_label_torch_mur2], dim=0)
#        print(audio_feature.shape)
        torch.autograd.set_detect_anomaly(True)
        audio_feature = audio_feature.permute(1,0)
#        audio_feature = audio_feature.unsqueeze(1)
        audio_feature[audio_feature != audio_feature] = 0
        audio_feature= torch.nan_to_num(audio_feature,nan=1e-6)+1e-6

        audio_feature = audio_feature.float().cuda()

        data_label_torch = data_label_torch.cuda()
        data_label_torch = data_label_torch.long()

#        print(audio_feature.shape)
        outputs = nnet(audio_feature).logits

#        print(type(outputs))
#        print(outputs)
        loss = criterion(outputs.float().squeeze(1), data_label_torch.float().squeeze(1))
#        print(outputs)
#        print(loss)
        
#        y_pred_softmax = softmax(outputs.float())
#        y_pred_softmax = F.log_softmax(outputs.float(), dim = 1)
#        _, output_tags = torch.max(outputs.float(), dim = 1)
#        _, output_tags = torch.max(y_pred_softmax, dim = 1)
#        print(outputs)
        output_tags = torch.ceil(outputs-0.5)
#        print(output_tags)
        output_tags = output_tags.detach().cpu().numpy()#.data.cpu().numpy()
        data_label_torch = data_label_torch.data.cpu().numpy()
        result = np.append(output_tags,data_label_torch,axis=1)
#        print('result : ',result)
        train_mur_f1 = f1_score(data_label_torch, output_tags)#, average='macro')



        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_epoch_mur_f1 += train_mur_f1.item()

        pre_train.extend(list(output_tags))
        label_train.extend(list(data_label_torch))
    pre_train[-1] = 0
    pre_train[-2] = 1
    label_train[-1] = np.array([0])
    label_train[-2] = np.array([1])
#    score_train = compute_weighted_accuracy(np.array(pre_train),np.array(label_train),'murmur')
    score_train =0
    logger.info("Iteration:{0}, loss = {1:.6f}, mur_f1 = {2:.3f}, mur_score = {3:.3f}".format(iter_, running_loss/all_file_train, train_epoch_mur_f1/all_file_train, score_train))

    with torch.no_grad():
        running_loss_dev = 0.0
        dev_epoch_f1 = 0.0


        pre_list = []
        label_list = []
        for audio_feature_dev, data_label_torch_dev  in tqdm(dataloader_dev) : 
            
            audio_feature_dev = audio_feature_dev.permute(1,0)
#            audio_feature_dev = audio_feature_dev.unsqueeze(1)
            audio_feature_dev[audio_feature_dev != audio_feature_dev] = 0
            audio_feature_dev = torch.nan_to_num(audio_feature_dev,nan=1e-6)+1e-6
            audio_feature_dev = audio_feature_dev.float().cuda()           
            label_dev = data_label_torch_dev

            label_dev = label_dev.cuda()
            label_dev = label_dev.long()

            outputs_dev = nnet(audio_feature_dev).logits
#            print(outputs_dev[0])

            loss_dev = criterion(outputs_dev.float().squeeze(1), label_dev.float().squeeze(1))
#            y_pred_softmax = softmax(outputs_dev.float())
#            y_pred_softmax = F.log_softmax(outputs_dev.float(), dim = 1)
#            _, output_tags = torch.max(outputs_dev.float(), dim = 1)
#            _, output_tags = torch.max(y_pred_softmax, dim = 1)
            output_tags = torch.ceil(outputs_dev-0.5)
            output_tags = output_tags.detach().cpu().numpy()#.data.cpu().numpy()
            label_dev = label_dev.data.cpu().numpy()

            dev_f1 = f1_score(label_dev, output_tags)#, average='macro')

            running_loss_dev += loss_dev.item()
            dev_epoch_f1 += dev_f1.item()
            pre_list.extend(list(output_tags))
            label_list.extend(list(label_dev))
        pre_list[-1] = 0
        pre_list[-2] = 1
#        pre_list[-3] = 2
        label_list[-1] = np.array([0])
        label_list[-2] = np.array([1])
#        label_list[-3] = np.array([2])
     
#        score_dev = compute_weighted_accuracy(np.array(pre_list),np.array(label_list),'murmur')
        score_dev = 0
        logger.info("Iteration:{0}, valid_loss = {1:.6f}, valid_mur_f1 = {2:.3f}, valid_mur_score = {3:.3f}".format(iter_, running_loss_dev/all_file_dev, dev_epoch_f1/all_file_dev, score_dev))





- 2545 training samples
- 80 training batches
- 639 testing samples
- 20 testing batches
Training model...


100%|███████████████████████████████████████████| 80/80 [01:26<00:00,  1.08s/it]

08-05 08:56 [MainProcess, 90738] [INFO ]  Iteration:0, loss = 0.427275, mur_f1 = 0.902, mur_score = 0.000
08-05 08:56 [MainProcess, 90738] [INFO ]  Iteration:0, loss = 0.427275, mur_f1 = 0.902, mur_score = 0.000



100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.91it/s]

08-05 08:56 [MainProcess, 90738] [INFO ]  Iteration:0, valid_loss = 0.427750, valid_mur_f1 = 0.897, valid_mur_score = 0.000
08-05 08:56 [MainProcess, 90738] [INFO ]  Iteration:0, valid_loss = 0.427750, valid_mur_f1 = 0.897, valid_mur_score = 0.000


...

08-05 09:24 [MainProcess, 90738] [INFO ]  Iteration:18, valid_loss = 0.447252, valid_mur_f1 = 0.944, valid_mur_score = 0.000
08-05 09:24 [MainProcess, 90738] [INFO ]  Iteration:18, valid_loss = 0.447252, valid_mur_f1 = 0.944, valid_mur_score = 0.000



100%|███████████████████████████████████████████| 80/80 [01:25<00:00,  1.07s/it]

08-05 09:25 [MainProcess, 90738] [INFO ]  Iteration:19, loss = 0.042682, mur_f1 = 0.990, mur_score = 0.000
08-05 09:25 [MainProcess, 90738] [INFO ]  Iteration:19, loss = 0.042682, mur_f1 = 0.990, mur_score = 0.000



100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.76it/s]

08-05 09:25 [MainProcess, 90738] [INFO ]  Iteration:19, valid_loss = 0.422546, valid_mur_f1 = 0.941, valid_mur_score = 0.000
08-05 09:25 [MainProcess, 90738] [INFO ]  Iteration:19, valid_loss = 0.422546, valid_mur_f1 = 0.941, valid_mur_score = 0.000


In [13]:
model_folder = 'w2v2_model'
outcome_classes = ['Abnormal','Normal']

In [14]:
save_challenge_model(model_folder, nnet,nnet, murmur_classes, outcome_classes, m_name = 'toy_ti', mel_shape = (100,313) )



In [15]:
def run_challenge_models(model, data, recordings, verbose,thres):
    torch.cuda.empty_cache()
#    if model['model'] == 'toy_ti' :
#        model1 = get_toy(model['mel_shape'])
#    model1.load_weights(model['model_fnm'])
    GPU_NUM = 4 # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
#    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device)


#    if model['model'] == 'toy_ti' :
#        trained_model =  torch.load(model['mur_model_fnm'])
#        trained_model2 =  torch.load(model['out_model_fnm'])
#        print(trained_model)
#        nnet = LCNN(mode = 'murmur')
#        nnet.load_state_dict(trained_model)
#        nnet = nnet.cuda()
#        nnet2 = LCNN(mode = 'murmur')
#        nnet2.load_state_dict(trained_model2)
#        nnet2 = nnet2.cuda()
        #print('*')
    nnet = model
    nnet2 = model
    nnet = nnet.cuda()
    nnet2 = nnet2.cuda()
    nnet.eval()
    nnet2.eval()
#    print(nnet)
        
#    murmur_classes = model['murmur_classes']
#    outcome_classes = model['outcome_classes']
    murmur_classes = ['Present','Unknown', 'Absent'] # 
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)

    # Load features.
    features = get_feature_one(data, verbose = 0)

    probab1 = np.empty((0,2))
    probab2 = np.empty((0,2))
    for i in range(len(recordings)) :
#        print(recordings[i])
        audio_feature,_ = feature_extract_raw(recordings[i]/ 32768,samp_sec=20)
#        print(type(audio_feature))
        audio_feature = torch.from_numpy(np.array(audio_feature))
        audio_feature[audio_feature != audio_feature] = 0
        audio_feature = torch.nan_to_num(audio_feature,nan=1e-6) +1e-6
        audio_feature = audio_feature.float().cuda()
#        audio_feature = audio_feature.unsqueeze(1)
#        audio_feature = audio_feature.permute(1,0,2)
        
#64, 1, 1, 157
        outputs = nnet(audio_feature).logits
        outputs2 = nnet2(audio_feature).logits
#        print(outputs)
#        print('outputs2 : ',outputs2)
#        softmax = nn.Softmax(dim = 1)
#        outputs_murmur = F.log_softmax(outputs.float(), dim = 1)
#        outputs_outcome = F.log_softmax(outputs2.float(), dim = 1)#
#        outputs_murmur = softmax(outputs)
#        outputs_outcome = softmax(outputs2)
        outputs_murmur = outputs
        outputs_outcome = outputs2
        outputs_murmur = torch.cat((1-outputs_murmur,outputs_murmur),dim = 1)
        outputs_outcome = torch.cat((1-outputs_outcome,outputs_outcome),dim = 1)
#        print('outputs_murmur : ',outputs_murmur)
        prob1 = outputs_murmur.mean(axis = 0).reshape(1,2) 
        prob1 = prob1.data.cpu().numpy()
        probab1 = np.concatenate((probab1,prob1),axis=0)

        prob2 = outputs_outcome.mean(axis = 0).reshape(1,2) 
        prob2 = prob2.data.cpu().numpy()
        probab2 = np.concatenate((probab2,prob2),axis=0)
        
    p1 = probab1.max(axis=0)
    p2 = probab2.max(axis=0)
    
        
    
#    print('p1 :',p1)
#    print('p2 :',p2)
        
        
    # Get classifier probabilities.
#    idx = np.argmax(p1)
    if p1[0]>thres:
        idx = 0
    else :
        idx = 2
#    idx2 = np.argmax(p2)
    if p2[0]>0.25 :
        idx2 = 0
    else :
        idx2 = 1

    n_p1 = np.zeros(len(murmur_classes))
    n_p1[0] = p1[0]
    n_p1[1] = 0
    n_p1[2] = p1[1]
    # Choose label with higher probability.
    labels_murmur = np.zeros(len(murmur_classes), dtype=np.int_)
    labels_murmur[idx] = 1
    labels_outcome = np.zeros(len(outcome_classes), dtype=np.int_)
    labels_outcome[idx2] = 1
    
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((labels_murmur, labels_outcome))
    probabilities = np.concatenate((n_p1, p2))

    return classes, labels, probabilities 

In [16]:
#!/usr/bin/env python

# Do *not* edit this script. Changes will be discarded so that we can process the models consistently.

# This file contains functions for running models for the 2022 Challenge. You can run it as follows:
#
#   python run_model.py model data outputs
#
# where 'model' is a folder containing the your trained model, 'data' is a folder containing the Challenge data, and 'outputs' is a
# folder for saving your model's outputs.

import numpy as np, os, sys
from helper_code import *
from team_code import load_challenge_model#, run_challenge_model
from tqdm import tqdm

# Run model.
def run_models(model_folder, data_folder, output_folder, allow_failures, verbose,thres):
    torch.cuda.empty_cache()
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')
        
    model = load_challenge_model(model_folder, verbose)
    
    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')
##########################
    # Iterate over the patient files.
    for i in tqdm(range(num_patient_files)):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_models(nnet, patient_data, recordings, verbose,thres) ### Teams: Implement this function!!!

        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')




In [17]:
model_folder = 'w2v2_model'
output_folder = 'w2v2_output'
allow_failures = False

In [36]:
run_models(model_folder, dev_folder, output_folder, allow_failures, verbose,thres= 0.5)

Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:15<00:00,  1.65s/it]

Done.


In [37]:
!python evaluate_model.py /Data1/physionet/data_split/murmur/test w2v2_output scores_w2v2.csv

In [38]:
!cat scores_w2v2.csv

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.783,0.636,0.483,0.733,0.765,14978.600

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.651,0.664,0.586,0.586,0.583,14216.054

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.957,0.500,0.893
AUPRC,0.883,0.073,0.953
F-measure,0.626,0.000,0.822
Accuracy,0.947,0.000,0.748

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.633,0.668
AUPRC,0.708,0.620
F-measure,0.591,0.582
Accuracy,0.582,0.591


# wav2vec2 threshold

In [18]:
import time
a = list(range(10,70,3))
for t in a :
    run_models(model_folder, dev_folder, output_folder, allow_failures, verbose,thres = t/100)
    time.sleep(2)
    print('threshold : ',t/100)
    !python evaluate_model.py /Data1/physionet/data_split/murmur/test w2v2_output scores_w2v2.csv
    !cat scores_w2v2.csv

Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:35<00:00,  1.76s/it]


Done.
threshold :  0.1
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.444,0.670,0.744,12959.510

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.556,0.000,0.774
Accuracy,0.974,0.000,0.655

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:39<00:00,  1.78s/it]


Done.
threshold :  0.13
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.447,0.675,0.747,13126.079

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.565,0.000,0.776
Accuracy,0.974,0.000,0.662

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710
Loading Challenge model...
Running model on Challenge data...


...

Done.
threshold :  0.61
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.498,0.759,0.768,15755.626

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.648,0.000,0.846
Accuracy,0.921,0.000,0.791

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:40<00:00,  1.78s/it]


Done.
threshold :  0.64
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.498,0.759,0.768,15755.626

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.648,0.000,0.846
Accuracy,0.921,0.000,0.791

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:42<00:00,  1.79s/it]


Done.
threshold :  0.67
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.502,0.764,0.771,15748.696

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.654,0.000,0.851
Accuracy,0.921,0.000,0.799

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710


In [19]:
run_models(model_folder, dev_folder, output_folder, allow_failures, verbose,thres= 0.7)
print('threshold : ',0.7)
!python evaluate_model.py /Data1/physionet/data_split/murmur/test w2v2_output scores_w2v2.csv
!cat scores_w2v2.csv

Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:56<00:00,  1.87s/it]


Done.
threshold :  0.7
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.509,0.775,0.776,15945.417

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.667,0.000,0.859
Accuracy,0.921,0.000,0.813

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710


In [20]:
run_models(model_folder, dev_folder, output_folder, allow_failures, verbose,thres= 0.8)
print('threshold : ',0.8)
!python evaluate_model.py /Data1/physionet/data_split/murmur/test w2v2_output scores_w2v2.csv
!cat scores_w2v2.csv

Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [05:59<00:00,  1.88s/it]


Done.
threshold :  0.8
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.522,0.791,0.784,16548.052

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.700,0.000,0.866
Accuracy,0.921,0.000,0.835

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710


In [22]:
run_models(model_folder, dev_folder, output_folder, allow_failures, verbose,thres= 0.9)
print('threshold : ',0.9)
!python evaluate_model.py /Data1/physionet/data_split/murmur/test w2v2_output scores_w2v2.csv
!cat scores_w2v2.csv

Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [06:00<00:00,  1.89s/it]


Done.
threshold :  0.9
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.533,0.806,0.792,16955.190

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.722,0.000,0.878
Accuracy,0.921,0.000,0.856

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710


In [23]:
run_models(model_folder, dev_folder, output_folder, allow_failures, verbose,thres= 0.93)
print('threshold : ',0.93)
!python evaluate_model.py /Data1/physionet/data_split/murmur/test w2v2_output scores_w2v2.csv
!cat scores_w2v2.csv

Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [06:01<00:00,  1.89s/it]


Done.
threshold :  0.93
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.541,0.817,0.798,16948.706

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.737,0.000,0.886
Accuracy,0.921,0.000,0.871

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710


In [24]:
import time
a = list(range(91,100,1))
for t in a :
    run_models(model_folder, dev_folder, output_folder, allow_failures, verbose,thres = t/100)
    time.sleep(2)
    print('threshold : ',t/100)
    !python evaluate_model.py /Data1/physionet/data_split/murmur/test w2v2_output scores_w2v2.csv
    !cat scores_w2v2.csv

Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [06:03<00:00,  1.90s/it]


Done.
threshold :  0.91
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.533,0.806,0.792,16955.190

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.722,0.000,0.878
Accuracy,0.921,0.000,0.856

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [06:02<00:00,  1.90s/it]


Done.
threshold :  0.92
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.533,0.806,0.792,16955.190

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.722,0.000,0.878
Accuracy,0.921,0.000,0.856

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710
Loading Challenge model...
Running model on Challenge data...


...

Done.
threshold :  0.97
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.561,0.838,0.798,18605.355

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.791,0.000,0.894
Accuracy,0.895,0.000,0.906

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710
Loading Challenge model...
Running model on Challenge data...


 73%|█████████████████████████████▊           | 139/191 [04:19<01:41,  1.95s/it]

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.566,0.843,0.801,18604.111

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.800,0.000,0.898
Accuracy,0.895,0.000,0.914

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710
Loading Challenge model...
Running model on Challenge data...


100%|█████████████████████████████████████████| 191/191 [06:02<00:00,  1.90s/it]


Done.
threshold :  0.99
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.782,0.635,0.570,0.848,0.803,18812.423

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.658,0.660,0.632,0.634,0.585,14408.661

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.949,0.500,0.898
AUPRC,0.874,0.073,0.958
F-measure,0.810,0.000,0.901
Accuracy,0.895,0.000,0.921

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.662,0.653
AUPRC,0.726,0.594
F-measure,0.611,0.653
Accuracy,0.561,0.710
